In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/OML---mini-project/src

In [ ]:
!pip install -r requirements.txt

In [ ]:
# This cell makes sure modules are auto-loaded when you change external python files
%load_ext autoreload
%autoreload 2

In [1]:
# Standard library imports
import math
import os

# Third party imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.optim.lr_scheduler as schedulers

# Local application imports
from utils import retrieve_setup, retrieve_training_params, training_loop

/Users/juangarcia/anaconda3/envs/ada/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/juangarcia/anaconda3/envs/ada/lib/python3.9/site-packages/torchvision/image.so, 6): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: /Users/juangarcia/anaconda3/envs/ada/lib/python3.9/site-packages/torchvision/image.so
  Expected in: /Users/juangarcia/anaconda3/envs/ada/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
 in /Users/juangarcia/anaconda3/envs/ada/lib/python3.9/site-packages/torchvision/image.so'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/juangarcia/anaconda3/envs/ada/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register

In [2]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "MobileNetV3Small"
dataset_name = "CIFAR10" # or "FashionMNIST"
n_epochs = 40
lr_max = 1e-3
lr_min = 1e-5

In [ ]:
# retrieves
model, dataset = retrieve_setup(model_name, dataset_name)
optimizer, _, loss_fn, _, batch_size = retrieve_training_params(model, dataset_name, "FixLR","parameters.yml")

# change lr from default to lr_max
for g in optimizer.param_groups:
    g["lr"] = lr_max

# implement linear increasing scheduler
total_iters = math.ceil(len(dataset["train"])/batch_size) * n_epochs
start_factor = lr_min/lr_max

scheduler_name, scheduler = "rangeLR", schedulers.LinearLR(
    optimizer,
    start_factor=start_factor,
    end_factor=1,
    total_iters=total_iters
)

In [ ]:
# run training loop
_, metrics = training_loop(
    model=model,
    dataset=dataset,
    scheduler=scheduler,
    optimizer=optimizer,
    loss_fn=loss_fn,
    n_epochs=n_epochs,
    batch_size=batch_size,
    train_strategy=("iter", 10),
    test_strategy=("iter", 10),
    scheduler_strategy="iter",
    file_name=f"/{dataset_name}_{scheduler_name}",
    device=device,
)